In [1]:
import nltk
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import re
from keras_preprocessing.sequence import pad_sequences
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Embedding

2023-03-01 16:47:05.664037: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-01 16:47:05.944859: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-01 16:47:05.944880: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-01 16:47:06.800377: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
data = pd.read_csv('datasets/hotel_data.csv')

In [3]:
data.head()

,area,city,country,crawl_date,highlight_value,hotel_overview,hotel_star_rating,image_urls,in_your_room,is_value_plus,...,property_type,qts,query_time_stamp,room_types,site_review_count,site_review_rating,sitename,state,traveller_rating,uniq_id
0,Hardasji Ki Magri,Udaipur,India,2016-06-21,{{facility}},|Zion Home Stay is located in a city that sets...,1 star,NaN,{{value}},no,...,Hotel,NaN,2016-06-21 04:06:50 +0000,NaN,/5,4.5,makemytrip,NaN,NaN,78ddf880bd7937d384ff278cc5b39d6e
1,Near Nai Gaon,Udaipur,India,2016-06-21,{{facility}},| Araliayas Resorts is a 3 star hotel located ...,3 star,NaN,{{value}},no,...,Hotel,NaN,2016-06-21 04:06:50 +0000,NaN,/5,4.5,makemytrip,NaN,NaN,9f9f9cbb2f7df8089b63d5cdeb257944
2,Near Bagore Ki Haveli,Udaipur,India,2016-06-21,{{facility}},|A 2 star property is located at 24 km from Ma...,2 star,NaN,{{value}},no,...,Hotel,NaN,2016-06-21 04:06:50 +0000,NaN,/5,NaN,makemytrip,NaN,NaN,b314bb7fa8bfb1ed306f517be21d729e
3,Dabok,Udaipur,India,2016-06-21,Airport Transfer|Car rental|Conference Hall|Cu...,|SNP House Airport Hotel And Restaurant is loa...,1 star,//imghtlak.mmtcdn.com/images/hotels/2014071815...,NaN,no,...,Hotel,NaN,2016-06-21 04:06:50 +0000,`standard,/5,NaN,makemytrip,NaN,NaN,e6f5bb3c2d76a78d978b9ceb0e31ec56
4,East Udaipur,Udaipur,India,2016-06-21,{{facility}},| Hotel Pichola Haveli is situated in the beau...,2 star,NaN,{{value}},no,...,Hotel,NaN,2016-06-21 04:06:50 +0000,NaN,/5,3.7,makemytrip,NaN,NaN,63072c301427b6ca450d31eea127bcf0


In [4]:
data.city.value_counts()

NewDelhiAndNCR    1163
Goa               1122
Mumbai             543
Jaipur             534
Bangalore          512
                  ... 
Kollur               1
Madla                1
Jeypore              1
Jispa                1
Haldia               1
Name: city, Length: 770, dtype: int64

In [5]:
data.shape

(20000, 33)

In [6]:
array=['Mumbai']
data=data.loc[data['city'].isin(array)]

In [7]:
data.head()

,area,city,country,crawl_date,highlight_value,hotel_overview,hotel_star_rating,image_urls,in_your_room,is_value_plus,...,property_type,qts,query_time_stamp,room_types,site_review_count,site_review_rating,sitename,state,traveller_rating,uniq_id
294,Charai,Mumbai,India,2016-08-28,Doctor on Call|Front desk|Laundry Service|Park...,"Nestled in Mumbai, a city with strong historic...",3,NaN,Bathroom Toiletries|Attached Bathroom|Hot & Co...,no,...,Hotel,2016-08-28 16:13:39 +0000,2016-08-28 16:13:39 +0000,Ac Superior Room|A/c Standard Rooms Double Occ...,NaN,NaN,makemytrip,Maharashtra,NaN,d78fae90ef2e1b5c2dfd547c61763a25
309,Andheri (East),Mumbai,India,2016-08-28,Air Conditioned|Airport Transfer|Conference Ha...,3 km from Chhatrapati Shivaji International Ai...,2,NaN,Bathroom Toiletries|Daily Newspaper|Kitchenett...,no,...,Hotel,2016-08-28 16:13:39 +0000,2016-08-28 16:13:39 +0000,Executive|Deluxe,NaN,NaN,makemytrip,Maharasta,Location:3.2/5 | Hospitality:3.1/5 | Facilitie...,030865f741982beb373efddecdc6d6c3
321,Khar,Mumbai,India,2016-08-28,Airport/Rlwy Stn Transfer|Bar|Conference Hall|...,Location Hotel Royal Garden is situated on Juh...,3,NaN,Electronic Safe|Bathroom Toiletries|Daily News...,no,...,Hotel,2016-08-28 16:13:39 +0000,2016-08-28 16:13:39 +0000,Superior Executive,NaN,NaN,makemytrip,Maharashtra,Location:4.5/5 | Hospitality:3.4/5 | Facilitie...,a1ced509350038775a7700ec67796bc2
334,Andheri (East),Mumbai,India,2016-08-28,24 Hour Check in-Icon|24 hour reception|24 hou...,City Guest House is a beautiful property locat...,2,NaN,Bathroom Toiletries|Hot/cold Water|Attached Ba...,no,...,Hotel,2016-08-28 16:13:39 +0000,2016-08-28 16:13:39 +0000,Standard Room|Deluxe Room|Triple Deluxe Room|S...,NaN,NaN,makemytrip,Maharashtra,Location:4.3/5 | Hospitality:3.8/5 | Facilitie...,f2820ae7707668ed6906bb227921f720
1238,Andheri (East),Mumbai,NaN,2016-08-22,24 Hour Check in-Icon|24 hour reception|24 hou...,Sai Residency Hotel is situated in the City of...,2,NaN,Bathroom Toiletries|Attached Bathroom|Hot & Co...,no,...,Hotel,2016-08-22 22:10:53 +0000,2016-08-22 22:10:53 +0000,Deluxe Dbl Air Cooled,NaN,NaN,makemytrip,MAHARASHTRA,NaN,b4af24952027ffbcd85a91bb6fe23f5d


In [8]:
data=data.hotel_overview

In [9]:
data.isna().sum()

53

In [10]:
data.dropna(inplace=True)
data.isna().sum()

0

In [11]:
data.iloc[0]

'Nestled in Mumbai, a city with strong historical links, wonderful British architecture, museums, beaches, places of worship, and above all, a true galaxy of stars where Bollywood reigns supreme. The Hotel Divya International is a delightful leisure with an absolute blend of service, charm and efficiency. The hotel offers facilities like front desk, parking, laundry, doctor on call and many more. It aims to extend the best possible hospitality experience to all its revered customers. The hotel is located at a distance of 350 m from S.T Workshop Bus Stop, 22.5 km from Chhatrapati Shivaji International Airport and 37.1 km from Chhatrapati Shivaji Terminus. The guests can head to prominent tourist attractions like Sanjay Gandhi National Park, KidZania Mumbai, Haji Ali Mosque, Iskon Temple, Shree Siddhivinayak Temple and many more. Shoppers can have fun filled shopping experience shopping at Viviana Mall(1.7 km). Food lovers can visit nearby restaurants to try local delights. The hotel has

In [12]:
stop=set(stopwords.words('english'))
print(stop)

{"doesn't", 'm', 'now', "hadn't", 'himself', 'so', 'having', 'than', 'while', 'do', "mightn't", 'am', 'ma', 'mustn', 've', 'before', 'can', 'that', 'above', 'here', 'won', 'each', 'y', 'other', 'have', 'ours', "you'd", 'yourself', 'at', 'they', "you're", 'such', 'them', 'those', "won't", "isn't", 'doesn', 'there', 'all', 'their', 'to', 'yourselves', 'what', 'will', 'it', 'again', 'because', "mustn't", 'or', 'should', 'been', 'just', 'did', "needn't", 'shan', "should've", 'more', 'has', 'until', "didn't", 'your', 'my', 'herself', 'needn', 'being', 'then', 'ain', 'hadn', "shan't", 'doing', 'her', 'i', 'for', 'nor', 'o', 'with', 'don', 'a', 'most', 'him', "haven't", 'had', 'he', 'yours', 'some', "don't", 'its', 'any', 'hasn', 'only', 'is', 'once', 'wasn', 'if', 'down', 'who', 'during', 'she', 'off', 'theirs', 's', 'between', 'where', 'couldn', 'were', 'further', 'over', 't', 'and', 'haven', 'are', 'of', 'but', 'both', 'too', "wasn't", 'isn', 'through', 'in', 'under', 'we', 'mightn', 'same

In [13]:
data.head()

294     Nestled in Mumbai, a city with strong historic...
309     3 km from Chhatrapati Shivaji International Ai...
321     Location Hotel Royal Garden is situated on Juh...
334     City Guest House is a beautiful property locat...
1238    Sai Residency Hotel is situated in the City of...
Name: hotel_overview, dtype: object

In [14]:
def stopwords_removal(data_point):
    data=[x for x in data_point.split() if x not in stop]
    return data

In [15]:
def clean_data(data):
    cleaned_data=[]
    all_unique_words_in_each_description=[]
    for entry in data:
        entry = re.sub(pattern='[^a-zA-Z]',repl=' ',string = entry)
        entry = re.sub(r'\b\w{0,1}\b', repl=' ',string = entry)
        entry = entry.lower()
        entry = stopwords_removal(entry)
        cleaned_data.append(entry)
        unique = list(set(entry))
        all_unique_words_in_each_description.extend(unique)
    return cleaned_data, all_unique_words_in_each_description

In [16]:
def unique_words(data):
    unique_words = set(data)
    return unique_words, len(unique_words)

In [17]:
cleaned_data, all_unique_words_in_each_description = clean_data(data)
unique_words, length_of_unique_words = unique_words(all_unique_words_in_each_description)

In [18]:
print(cleaned_data[0])

['nestled', 'mumbai', 'city', 'strong', 'historical', 'links', 'wonderful', 'british', 'architecture', 'museums', 'beaches', 'places', 'worship', 'true', 'galaxy', 'stars', 'bollywood', 'reigns', 'supreme', 'hotel', 'divya', 'international', 'delightful', 'leisure', 'absolute', 'blend', 'service', 'charm', 'efficiency', 'hotel', 'offers', 'facilities', 'like', 'front', 'desk', 'parking', 'laundry', 'doctor', 'call', 'many', 'aims', 'extend', 'best', 'possible', 'hospitality', 'experience', 'revered', 'customers', 'hotel', 'located', 'distance', 'workshop', 'bus', 'stop', 'km', 'chhatrapati', 'shivaji', 'international', 'airport', 'km', 'chhatrapati', 'shivaji', 'terminus', 'guests', 'head', 'prominent', 'tourist', 'attractions', 'like', 'sanjay', 'gandhi', 'national', 'park', 'kidzania', 'mumbai', 'haji', 'ali', 'mosque', 'iskon', 'temple', 'shree', 'siddhivinayak', 'temple', 'many', 'shoppers', 'fun', 'filled', 'shopping', 'experience', 'shopping', 'viviana', 'mall', 'km', 'food', 'lo

In [19]:
length_of_unique_words

3395

In [20]:
def build_indices(unique_words):
    word_to_idx = {}
    idx_to_word = {}
    for i, word in enumerate(unique_words):
        word_to_idx[word] = i
        idx_to_word[i] = word
    return word_to_idx, idx_to_word

In [21]:
word_to_idx, idx_to_word = build_indices(unique_words)
print(idx_to_word)

{0: 'mumbais', 1: 'ratnagiri', 2: 'reflect', 3: 'parle', 4: 'technical', 5: 'bottle', 6: 'weighing', 7: 'important', 8: 'window', 9: 'comfortable', 10: 'stepping', 11: 'excellently', 12: 'recreation', 13: 'size', 14: 'mesmerizing', 15: 'walks', 16: 'sober', 17: 'driven', 18: 'hills', 19: 'arts', 20: 'gadgets', 21: 'japanese', 22: 'neutrals', 23: 'areas', 24: 'dose', 25: 'districts', 26: 'stardust', 27: 'charismatic', 28: 'massages', 29: 'architectures', 30: 'government', 31: 'sit', 32: 'visiting', 33: 'includes', 34: 'fabulously', 35: 'oritel', 36: 'synagouge', 37: 'colourful', 38: 'temples', 39: 'ice', 40: 'phone', 41: 'bustle', 42: 'anchor', 43: 'nationalities', 44: 'chowpatty', 45: 'covers', 46: 'populous', 47: 'comes', 48: 'exclusivity', 49: 'years', 50: 'future', 51: 'possible', 52: 'excursions', 53: 'regal', 54: 'parkway', 55: 'availing', 56: 'big', 57: 'boxes', 58: 'fulfil', 59: 'east', 60: 'exclusively', 61: 'blended', 62: 'session', 63: 'magnet', 64: 'orchid', 65: 'additional'

In [22]:
def prepare_corpus(corpus, word_to_idx):
    
    sequences = []
    for line in corpus:
        tokens = line
        for i in range(1, len(tokens)):
            i_gram_sequence = tokens[:i+1]
            i_gram_sequence_ids = []
            
            for j, token in enumerate(i_gram_sequence):
                i_gram_sequence_ids.append(word_to_idx[token])
                
            sequences.append(i_gram_sequence_ids)
    
    return sequences

In [23]:
sequences = prepare_corpus(cleaned_data, word_to_idx)
max_sequence_len = max([len(x) for x in sequences])

In [24]:
max_sequence_len

308

In [25]:
print(sequences[0])
print(sequences[1])

[2171, 170]
[2171, 170, 1034]


In [26]:
print(idx_to_word[1647])
print(idx_to_word[867])
print(idx_to_word[1452])

heads
hot
awadhi


In [27]:
len(sequences)

51836

In [28]:
def build_input_data(sequences, max_sequence_len, length_of_unique_words):
    sequences = np.array(pad_sequences(sequences, maxlen = max_sequence_len, padding = 'pre'))
    X = sequences[:,:-1]
    y = sequences[:,-1]
    y = np_utils.to_categorical(y, length_of_unique_words)
    return X, y

In [29]:
length_of_unique_words

3395

In [30]:
X, y = build_input_data(sequences, max_sequence_len, length_of_unique_words)

In [31]:
def create_model(max_sequence_len, length_of_unique_words):
    model = Sequential()
    model.add(Embedding(length_of_unique_words, 10, input_length=max_sequence_len - 1))
    model.add(LSTM(128))
    model.add(Dropout(0.2))
    model.add(Dense(length_of_unique_words, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model

In [32]:
model = create_model(max_sequence_len, length_of_unique_words)
model.summary()

2023-03-01 16:47:15.494090: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-01 16:47:15.494315: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-01 16:47:15.494388: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-03-01 16:47:15.494434: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-03-01 16:47:15.494477: W tensorflow/c

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 307, 10)           33950     
                                                                 
 lstm (LSTM)                 (None, 128)               71168     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 3395)              437955    
                                                                 
Total params: 543,073
Trainable params: 543,073
Non-trainable params: 0
_________________________________________________________________


In [33]:
model.fit(X, y, batch_size = 512, epochs=10)

Epoch 1/10


2023-03-01 16:47:36.333570: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 703932880 exceeds 10% of free system memory.


102/102 [==============================] - 84s 805ms/step - loss: 6.9221
Epoch 2/10
102/102 [==============================] - 84s 820ms/step - loss: 6.5690
Epoch 3/10
102/102 [==============================] - 85s 833ms/step - loss: 6.5010
Epoch 4/10
102/102 [==============================] - 89s 868ms/step - loss: 6.4430
Epoch 5/10
102/102 [==============================] - 81s 793ms/step - loss: 6.3646
Epoch 6/10
102/102 [==============================] - 79s 779ms/step - loss: 6.2614
Epoch 7/10
102/102 [==============================] - 86s 845ms/step - loss: 6.2107
Epoch 8/10
102/102 [==============================] - 87s 856ms/step - loss: 6.0847
Epoch 9/10
102/102 [==============================] - 95s 936ms/step - loss: 5.9550
Epoch 10/10
102/102 [==============================] - 102s 1000ms/step - loss: 5.8478


In [34]:
model_structure = model.to_json()
with open("model/text_generation_using_LSTM.json", "w") as json_file:
    json_file.write(model_structure)
model.save_weights("model/text_generation_using_LSTM.h5")

In [45]:
def generate_text(seed_text, next_words, model, max_seq_len):
    for _ in range(next_words):
        cleaned_data = clean_data([seed_text])
        sequences= prepare_corpus(cleaned_data[0], word_to_idx)
        sequences = pad_sequences([sequences[-1]], maxlen=max_seq_len-1, padding='pre')
        predicted_x = model.predict(sequences, verbose=0)
        predicted=np.argmax(predicted_x,axis=1)
        output_word = ''
        output_word = idx_to_word[predicted[0]]            
        seed_text = seed_text + " " + output_word
        
    return seed_text.title()

In [46]:
print(generate_text("in Mumbai there we need", 30, model, max_sequence_len))

In Mumbai There We Need Mumbai Mumbai Mumbai Mumbai Mumbai Mumbai Mumbai Mumbai Hotel Hotel Offers Mumbai Hotel Located Mumbai Hotel Offers Hotel Hotel Offers Mumbai Hotel Hotel Offers Mumbai Hotel Hotel Located Mumbai Hotel


In [49]:
print(generate_text("Best Mumbai", 30, model, max_sequence_len))

Best Mumbai Mumbai Mumbai Mumbai Mumbai Mumbai Mumbai Mumbai Mumbai Mumbai Hotel Hotel Offers Mumbai Hotel Located Mumbai Hotel Offers Hotel Hotel Offers Mumbai Hotel Hotel Offers Mumbai Hotel Hotel Located Mumbai


In [ ]:
!git add TextGenerationLSTM.ipynb
!git comm